In [10]:
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
import sagemaker
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, rule_configs

role = get_execution_role()

region = boto3.Session().region_name

sagemaker_session = sagemaker.Session()

bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-tripfare'

In [27]:
!python -m pip install -Uq pip
!python -m pip install -q awswrangler sagemaker==2.59.5 boto3==1.18.51

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [7]:
%store
%store -r

Stored variables and their in-db values:
test_path                   -> 's3://sagemaker-us-east-1-631450739534/sagemaker/D
train_path                  -> 's3://sagemaker-us-east-1-631450739534/sagemaker/D
validation_path             -> 's3://sagemaker-us-east-1-631450739534/sagemaker/D


In [21]:
# training step for generating model artifacts
training_instance_type = "ml.m5.xlarge"
model_output = f"s3://{bucket}/{prefix}/model"

# Define the XGBoost training report rules
# see: https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-training-xgboost-report.html
rules = [Rule.sagemaker(rule_configs.create_xgboost_report())]

image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.3-1",
    py_version="py3",
    instance_type=training_instance_type,
)
xgb_train = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=2,
    output_path=model_output,
    base_job_name=f"{prefix.split('/')[-1]}-train",
    sagemaker_session=sagemaker_session,
    role=role,
    disable_profiler=False,  # Profile processing job
    rules=rules,  # Report processing job
)

In [22]:
# Set some hyper parameters
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgb_train.set_hyperparameters(
    objective="reg:squarederror",
    num_round=100,
    early_stopping_rounds=10,
    max_depth=9,
    eta=0.2,
    gamma=4,
    min_child_weight=300,
    subsample=0.8,
)

In [23]:
from sagemaker.inputs import TrainingInput
content_type = "csv"
train_input = TrainingInput(
    train_path, content_type=content_type, distribution='ShardedByS3Key'
)
validation_input = TrainingInput(
    validation_path, content_type=content_type
)

In [24]:
xgb_train.fit({'train': train_input, 'validation': validation_input})

2021-09-30 03:08:58 Starting - Starting the training job...
2021-09-30 03:09:22 Starting - Launching requested ML instancesCreateXgboostReport: InProgress
ProfilerReport-1632971338: InProgress
......
2021-09-30 03:10:22 Starting - Preparing the instances for training............
2021-09-30 03:12:22 Downloading - Downloading input data...
2021-09-30 03:12:56 Training - Training image download completed. Training in progress..[2021-09-30 03:12:57.921 ip-10-2-81-62.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-09-30:03:12:58:INFO] Imported framework sagemaker_xgboost_container.training
[2021-09-30:03:12:58:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2021-09-30:03:12:58:INFO] No GPUs detected (normal if no gpus installed)
[2021-09-30:03:12:58:INFO] Running XGBoost Sagemaker in algorithm mode
[2021-09-30:03:12:58:INFO] Determined delimiter of CSV input is ','
[2021-09-30:03:12:58:INFO] Determined del

In [112]:
training_job_name = xgb_train.latest_training_job.job_name
model_url = xgb_train.model_data

In [113]:
%store training_job_name
%store model_url

Stored 'training_job_name' (str)
Stored 'model_url' (str)


In [25]:
xgboost_endpoint_name = "xgboost-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
xgboost_predictor = xgb_train.deploy(
    initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=xgboost_endpoint_name
)

In [33]:
import awswrangler as wr
test_df = wr.s3.read_csv(
        path=test_path, dataset=True, nrows=50, header=None
    )

In [91]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

xgboost_predictor.serializer = CSVSerializer()
xgboost_predictor.deserializer = CSVDeserializer()
xgboost_predictor.predict(test_df.iloc[:,1:].values)[0]

['18.535001754760742',
 '11.761689186096191',
 '15.242060661315918',
 '10.931342124938965',
 '28.7455997467041',
 '5.776196479797363',
 '17.04107093811035',
 '23.121328353881836',
 '23.43851089477539',
 '24.959461212158203',
 '17.03936195373535',
 '10.529860496520996',
 '13.59338092803955',
 '15.18577766418457',
 '29.108205795288086',
 '21.132905960083008',
 '10.574868202209473',
 '10.79728889465332',
 '7.7672014236450195',
 '16.365497589111328',
 '28.37923240661621',
 '7.599057197570801',
 '17.271512985229492',
 '11.84465503692627',
 '7.650399208068848',
 '16.048227310180664',
 '12.509238243103027',
 '24.174177169799805',
 '22.60703468322754',
 '17.289512634277344',
 '10.817380905151367',
 '16.75693130493164',
 '6.5838398933410645',
 '16.780078887939453',
 '6.483026504516602',
 '7.807983875274658',
 '15.202792167663574',
 '21.334579467773438',
 '22.63629150390625',
 '15.02584171295166',
 '32.15640640258789',
 '8.679388999938965',
 '20.924877166748047',
 '3.613661527633667',
 '7.089037

## Automatic model Tuning 
Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose.
For example, suppose that you want to solve a binary classification problem on this marketing dataset. Your goal is to maximize the area under the curve (auc) metric of the algorithm by training an XGBoost Algorithm model. You don't know which values of the eta, alpha, min_child_weight, and max_depth hyperparameters to use to train the best model. To find the best values for these hyperparameters, you can specify ranges of values that Amazon SageMaker hyperparameter tuning searches to find the combination of values that results in the training job that performs the best as measured by the objective metric that you chose. Hyperparameter tuning launches training jobs that use hyperparameter values in the ranges that you specified, and returns the training job with highest auc.


In [115]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}

In [126]:
objective_metric_name = 'validation:rmse'
objective_type = 'Minimize'

In [127]:
tuner = HyperparameterTuner(xgb_train,
                            objective_metric_name,
                            hyperparameter_ranges,
                            objective_type=objective_type,
                            max_jobs=10,
                            max_parallel_jobs=3)

In [128]:
tuner.fit({'train': train_input, 'validation': validation_input})

............................................................................................................................................................................................................................................................................................!


In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
# return the best training job name
tuner.best_training_job()

In [ ]:
#  Deploy the best trained or user specified model to an Amazon SageMaker endpoint
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

In [ ]:
# Create a serializer
tuner_predictor.serializer = CSVSerializer()
tuner_predictor.deserializer = CSVDeserializer()

In [ ]:
# Predict
tuner_predictor.predict(test_df.iloc[:,1:].values)[0]

### (Optional) Delete the Endpoint
If you're done with this exercise, please run the delete_endpoint line in the cell below. This will remove the hosted endpoint and avoid any charges from a stray instance being left on.

In [114]:
xgboost_predictor.delete_endpoint(delete_endpoint_config=True)

In [ ]:
tuner_predictor.delete_endpoint(delete_endpoint_config=True)